In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

data/sample_submission.csv
data/test_identity.csv
data/test_transaction.csv
data/train_identity.csv
data/train_transaction.csv


In [3]:
%%time
train_identity_df = pd.read_csv('data/train_identity.csv',index_col='TransactionID')
train_transaction_df = pd.read_csv('data/train_transaction.csv',index_col='TransactionID')
# test_identity_df = pd.read_csv('data/test_identity.csv',index_col='TransactionID')
# test_transaction_df = pd.read_csv('data/test_transaction.csv',index_col='TransactionID')

CPU times: user 23.6 s, sys: 3.85 s, total: 27.5 s
Wall time: 25.6 s


In [4]:
print("Train Identity DF Length: {} Features: {}".format(train_identity_df.shape[0],train_identity_df.shape[1]))
print("Train Transaction DF Length: {} Features: {}".format(train_transaction_df.shape[0],train_transaction_df.shape[1]))
# print("Test Identity DF Length: {} Features: {}".format(test_identity_df.shape[0],test_identity_df.shape[1]))
# print("Test Transaction DF Length: {} Features: {}".format(test_transaction_df.shape[0],test_transaction_df.shape[1]))


Train Identity DF Length: 144233 Features: 40
Train Transaction DF Length: 590540 Features: 393


In [5]:
train_df = train_transaction_df.merge(train_identity_df,how='left',left_index=True, right_index=True)
del train_identity_df, train_transaction_df
gc.collect() 

21

In [6]:
train_df.shape

(590540, 433)

In [7]:
train_target = train_df['isFraud'].copy()
train_df_X = train_df.drop('isFraud', axis=1)
gc.collect() 

0

In [8]:
train_target.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [11]:
20663/(20663+569877) * 100
569877/20663

27.579586700866283

In [23]:
#Helper functions
def value_counts(df,keepna: bool):
    return df.value_counts(dropna= not keepna)
def null_count(df):
    return df.isnull().sum(axis = 0)

***Categorical Variables***

In [10]:
categorical_variables = ['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain',
                         'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 
                         'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 
                         'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
len(categorical_variables)

49

In [105]:
train_df_X_categorical = train_df_X[categorical_variables]
train_df_X_categorical.head()

,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,outlook.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,yahoo.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,gmail.com,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


***Impute Null Values of Categorical Variables using custom values***

In [149]:
catgorical_fillna = {
    'ProductCD': 'others',
    'P_emaildomain': 'others.others',
    'R_emaildomain': 'others.others',
    'card4': 'others',
    'card6': 'others',
    'M1': 'others',
    'M2': 'others',
    'M3': 'others',
    'M4': 'others',
    'M5': 'others',
    'M6': 'others',
    'M7': 'others',
    'M8': 'others',
    'M9': 'others',
    'id_12': 'others',
    'id_15': 'others',
    'id_16': 'others',
    'id_23': 'others',
    'id_27': 'others',
    'id_28': 'others',
    'id_29': 'others',
    'id_30': 'others',
    'id_31': 'others',
    'id_34': 'others',
    'id_35': 'others',
    'id_36': 'others',
    'id_37': 'others',
    'id_38': 'others',
    'DeviceType': 'others',
    'DeviceInfo': 'others',
    'card1': -9999,
    'card2': -9999,
    'card3': -9999,
    'card5': -9999,
    'addr1': -9999,
    'addr2': -9999,
    'id_13': -9999,
    'id_14': -9999,
    'id_17': -9999,
    'id_18': -9999,
    'id_19': -9999,
    'id_20': -9999,
    'id_21': -9999,
    'id_22': -9999,
    'id_24': -9999,
    'id_25': -9999,
    'id_26': -9999,
    'id_32': -9999,
    'id_33': -9999,
}
print(len(catgorical_fillna.keys()))
train_df_X_categorical_imputed = train_df_X_categorical.fillna(catgorical_fillna)
train_df_X_categorical_imputed.head()

49


,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,W,13926,-9999.0,150.0,discover,142.0,credit,315.0,87.0,others.others,...,others,-9999.0,-9999,others,others,others,others,others,others,others
2987001,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,...,others,-9999.0,-9999,others,others,others,others,others,others,others
2987002,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,outlook.com,...,others,-9999.0,-9999,others,others,others,others,others,others,others
2987003,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,yahoo.com,...,others,-9999.0,-9999,others,others,others,others,others,others,others
2987004,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,gmail.com,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [150]:
def split_email_id(email_id: str):
    email_id_split = email_id.split('.')
    return email_id_split
train_df_X_categorical_imputed['P_emaildomain_provider'] = train_df_X_categorical_imputed['P_emaildomain'].map(lambda x: split_email_id(x)[0])
train_df_X_categorical_imputed['P_emaildomain_extension'] = train_df_X_categorical_imputed['P_emaildomain'].map(lambda x: split_email_id(x)[-1])
train_df_X_categorical_imputed[['P_emaildomain_provider','P_emaildomain_extension']].head()

,P_emaildomain_provider,P_emaildomain_extension
TransactionID,,
2987000,others,others
2987001,gmail,com
2987002,outlook,com
2987003,yahoo,com
2987004,gmail,com


In [151]:
train_df_X_categorical_imputed['P_emaildomain_extension'].value_counts()

com       466477
others     94456
net        24542
mx          2499
es           877
de           506
gmail        496
fr           494
uk           161
jp            32
Name: P_emaildomain_extension, dtype: int64

In [152]:
train_df_X_categorical_imputed['R_emaildomain_provider'] = train_df_X_categorical_imputed['R_emaildomain'].map(lambda x: split_email_id(x)[0])
train_df_X_categorical_imputed['R_emaildomain_extension'] = train_df_X_categorical_imputed['R_emaildomain'].map(lambda x: split_email_id(x)[-1])
train_df_X_categorical_imputed[['R_emaildomain_provider','R_emaildomain_extension']].head()

,R_emaildomain_provider,R_emaildomain_extension
TransactionID,,
2987000,others,others
2987001,others,others
2987002,others,others
2987003,others,others
2987004,others,others


In [153]:
train_df_X_categorical_imputed['R_emaildomain_extension'].value_counts()

others    453249
com       127870
net         4849
mx          2469
es           782
de           501
fr           485
uk           144
gmail         95
edu           63
jp            33
Name: R_emaildomain_extension, dtype: int64

In [154]:
train_df_X_categorical_imputed = train_df_X_categorical_imputed.drop(['R_emaildomain','P_emaildomain'], axis=1)
gc.collect()

815

In [155]:
# From https://www.kaggle.com/pavelvpster/ieee-fraud-eda-lightgbm-baseline

from sklearn.preprocessing import LabelEncoder
def encode_categorial_features_fit(df):
    categorical_encoders = dict()
    for c in df.columns:
        encoder = LabelEncoder()
        encoder.fit(df[c].astype(str).values)
        categorical_encoders[c] = encoder
    return categorical_encoders

def encode_categorial_features_transform(df, encoders):
    out = pd.DataFrame(index=df.index)
    for c in encoders.keys():
        if c in df.columns:
            out[c] = encoders[c].transform(df[c].astype(str).values)
    return out

In [161]:
train_df_X_categorical_encoded = encode_categorial_features_transform(train_df_X_categorical_imputed, encode_categorial_features_fit(train_df_X_categorical_imputed))
gc.collect()

301

In [162]:
train_df_X_categorical_encoded.head()

,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,M1,...,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_emaildomain_provider,P_emaildomain_extension,R_emaildomain_provider,R_emaildomain_extension
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,4,3417,0,43,1,39,1,167,66,1,...,2,2,2,2,2,1743,27,8,27,9
2987001,4,7922,304,43,2,3,1,174,66,2,...,2,2,2,2,2,1743,15,0,27,9
2987002,4,9383,390,43,4,59,2,179,66,1,...,2,2,2,2,2,1743,28,0,27,9
2987003,4,6991,467,43,2,15,2,283,66,2,...,2,2,2,2,2,1743,43,0,27,9
2987004,1,9262,414,43,2,3,1,242,66,2,...,1,0,1,1,1,954,15,0,27,9


In [163]:
def df_categorical_variable_summary(df):
    least = df.min().to_frame('Least')
    highest = df.max().to_frame('Highest')
    mode = df.mode().transpose()
    mode.columns = ['Mode']
    categorical_variable_summary = least.join(highest)\
                                        .join(mode)
    categorical_variable_summary['Variable Name'] = categorical_variable_summary.index
    categorical_variable_summary['# of Levels'] = categorical_variable_summary['Highest'] - categorical_variable_summary['Least'] + 1
    categorical_variable_summary = categorical_variable_summary[['Variable Name','# of Levels','Least','Highest','Mode']]
    return categorical_variable_summary


In [164]:
df_categorical_variable_summary(train_df_X_categorical_encoded).head()

,Variable Name,# of Levels,Least,Highest,Mode
ProductCD,ProductCD,5,0,4,4
card1,card1,13553,0,13552,11915
card2,card2,501,0,500,221
card3,card3,115,0,114,43
card4,card4,5,0,4,4


In [165]:
df_categorical_variable_summary(train_df_X_categorical_encoded).shape[0]

51

***Continuous Variables***

In [170]:
continuous_variables = [col for col in train_df_X.columns if col not in categorical_variables]
len(continuous_variables)

383

***Pre-Imputation Continuous Variables Distribution***

In [181]:
def df_continuous_variable_summary(df, continuous_variable: list):
    continuous_features_df = df[continuous_variable]
    continuous_features_summary = continuous_features_df.describe().transpose()
    continuous_features_summary['variable'] = continuous_features_summary.index
    continuous_features_kurtosis = continuous_features_df.kurtosis().to_frame('kurtosis')
    continuous_features_kurtosis['variable'] = continuous_features_kurtosis.index
    continuous_features_skewness = continuous_features_df.skew().to_frame('skewness')
    continuous_features_skewness['variable'] = continuous_features_skewness.index
    continuous_data_summary = continuous_features_summary.merge(continuous_features_skewness, on='variable')\
                                                     .merge(continuous_features_kurtosis, on='variable')
    continuous_data_summary["NA's"] = continuous_features_df.shape[0] - continuous_data_summary['count']
    continuous_data_summary = continuous_data_summary.drop('count', axis=1)
    continuous_data_summary['Skewed (Y/N)'] = ['N' if skew<=0.5 and skew>=-0.5 else 'Y' for skew in continuous_data_summary['skewness']]
    continuous_data_summary = continuous_data_summary[['variable','min','25%','50%','mean','75%','max',"NA's",'Skewed (Y/N)','skewness','kurtosis']]
    return continuous_data_summary
    

In [183]:
continuous_data_summary = df_continuous_variable_summary(train_df_X, continuous_variables)
continuous_data_summary.to_csv('continuous_data_summary.csv',sep='\t', index=False)
gc.collect()

478

In [191]:
train_df_X_continuous = train_df_X[continuous_variables]

In [189]:
# from sklearn.impute import SimpleImputer

# def impute_continuous_features_fit(df):
#     continuous_imputers = dict()
#     for c in df.columns:
#         imputer = SimpleImputer( strategy='mean')
#         imputer.fit(df[c])
#         continuous_imputers[c] = imputer
#     return continuous_imputers

# def impute_continuous_features_transform(df, imputers):
#     out = pd.DataFrame(index=df.index)
#     for c in imputers.keys():
#         if c in df.columns:
#             out[c] = imputers[c].transform(df[c])
#     return out

In [192]:
# train_df_X_continuous_imputed = impute_continuous_features_transform(train_df_X_continuous, impute_continuous_features_fit(train_df_X_continuous))

ValueError: Expected 2D array, got 1D array instead:
array=[   86400.    86401.    86469. ... 15811079. 15811088. 15811131.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [224]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(train_df_X_continuous)
train_array_X_continuous = imp_mean.transform(train_df_X_continuous)

In [225]:
train_df_X_continuous_imputed = pd.DataFrame(train_array_X_continuous, columns=train_df_X_continuous.columns, index=train_df_X_continuous.index)

In [226]:
train_df_X_continuous_imputed.head()

,TransactionDT,TransactionAmt,dist1,dist2,C1,C2,C3,C4,C5,C6,...,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,86400.0,68.5,19.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0,...,125800.5,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
2987001,86401.0,29.0,8.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0,...,125800.5,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
2987002,86469.0,59.0,287.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0,...,125800.5,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
2987003,86499.0,50.0,8.0,37.0,2.0,5.0,0.0,0.0,0.0,4.0,...,125800.5,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0
2987004,86506.0,50.0,8.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0,...,70787.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0


In [241]:
#https://www.kaggle.com/pavelvpster/ieee-fraud-feature-selection-rfecv/comments
params = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'boost_from_average': True,
    'num_threads': 4,
    
    'num_leaves': 500,
    # 'min_data_in_leaf': 25,
    'max_depth': -1,
    'learning_rate': 0.01
}

In [228]:
X_train = train_df_X_continuous_imputed.join(train_df_X_categorical_encoded)
y_train = train_target

In [229]:
y_train.head()

TransactionID
2987000    0
2987001    0
2987002    0
2987003    0
2987004    0
Name: isFraud, dtype: int64

In [230]:
#https://www.kaggle.com/pavelvpster/ieee-fraud-feature-selection-rfecv/comments

from sklearn.model_selection import train_test_split
percent = 25 # percent of dataset to use for feature selection

x_train_train, x_train_valid, y_train_train, y_train_valid = train_test_split(X_train, y_train, test_size=1.0-percent/100.0, random_state=42)

# del train_target
del X_train
del y_train

gc.collect()

244

In [231]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=x_train_train, label=y_train_train)
dvalid = xgb.DMatrix(data=x_train_valid, label=y_train_valid)
evallist = [(dvalid, 'eval'), (dtrain, 'train')]
num_rounds = 10
early_stopping_rounds = 5
gc.collect()

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


14

In [232]:
xgb_model = xgb.train(params,dtrain,num_rounds,evallist,early_stopping_rounds=early_stopping_rounds)
y_train_valid_pred = xgb_model.predict(dvalid)

[0]	eval-error:0.031368	train-error:0.031036
Multiple eval metrics have been passed: 'train-error' will be used for early stopping.

Will train until train-error hasn't improved in 5 rounds.
[1]	eval-error:0.031368	train-error:0.031036
[2]	eval-error:0.031368	train-error:0.031036
[3]	eval-error:0.031368	train-error:0.031036
[4]	eval-error:0.031368	train-error:0.031036
[5]	eval-error:0.031368	train-error:0.031036
Stopping. Best iteration:
[0]	eval-error:0.031368	train-error:0.031036



In [234]:
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support
score_all_features = roc_auc_score(y_train_valid, y_train_valid_pred)
print('AUC:', score_all_features)
print(precision_recall_fscore_support(y_true=y_train_valid, y_pred=y_train_valid_pred>0.5))

AUC: 0.639163109447623
(array([0.96930617, 0.84454659]), array([0.99912955, 0.13003474]), array([0.98399193, 0.22536939]), array([427363,  15542]))


***LightGBM Model***

In [242]:
#https://www.kaggle.com/pavelvpster/ieee-fraud-feature-selection-rfecv/comments
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(**params).fit(x_train_train, y_train_train)
y_all_features = lgb_model.predict_proba(x_train_valid)[:,1]
score_all_features = roc_auc_score(y_train_valid, y_all_features)
print('Score / all features:', score_all_features)

Score / all features: 0.9170710608303864


In [243]:
from sklearn.feature_selection import RFECV
feature_selector = RFECV(lgb.LGBMClassifier(**params), step=10, scoring='roc_auc', cv=5, verbose=1)
feature_selector.fit(x_train_train, y_train_train)
print('Features selected:', feature_selector.n_features_)

Fitting estimator with 434 features.
Fitting estimator with 424 features.
Fitting estimator with 414 features.
Fitting estimator with 404 features.
Fitting estimator with 394 features.
Fitting estimator with 384 features.
Fitting estimator with 374 features.
Fitting estimator with 364 features.
Fitting estimator with 354 features.
Fitting estimator with 344 features.
Fitting estimator with 334 features.
Fitting estimator with 324 features.
Fitting estimator with 314 features.
Fitting estimator with 304 features.
Fitting estimator with 294 features.
Fitting estimator with 284 features.
Fitting estimator with 274 features.
Fitting estimator with 264 features.
Fitting estimator with 254 features.
Fitting estimator with 244 features.
Fitting estimator with 234 features.
Fitting estimator with 224 features.
Fitting estimator with 214 features.
Fitting estimator with 204 features.
Fitting estimator with 194 features.
Fitting estimator with 184 features.
Fitting estimator with 174 features.
F

In [244]:
selected_features = [f for f in x_train_train.columns[feature_selector.ranking_ == 1]]

selected_features

['TransactionDT',
 'TransactionAmt',
 'dist1',
 'dist2',
 'C1',
 'C2',
 'C4',
 'C5',
 'C6',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D13',
 'D14',
 'D15',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V11',
 'V12',
 'V13',
 'V19',
 'V20',
 'V24',
 'V29',
 'V35',
 'V36',
 'V37',
 'V38',
 'V45',
 'V48',
 'V53',
 'V54',
 'V55',
 'V56',
 'V61',
 'V62',
 'V66',
 'V67',
 'V70',
 'V75',
 'V76',
 'V77',
 'V78',
 'V82',
 'V83',
 'V86',
 'V87',
 'V91',
 'V96',
 'V97',
 'V99',
 'V102',
 'V124',
 'V126',
 'V127',
 'V128',
 'V129',
 'V130',
 'V131',
 'V132',
 'V133',
 'V135',
 'V136',
 'V137',
 'V139',
 'V159',
 'V160',
 'V164',
 'V165',
 'V166',
 'V172',
 'V206',
 'V209',
 'V212',
 'V216',
 'V221',
 'V228',
 'V233',
 'V234',
 'V258',
 'V263',
 'V264',
 'V265',
 'V273',
 'V280',
 'V281',
 'V282',
 'V283',
 'V285',
 'V287',
 'V289',
 'V291',
 'V292',
 'V293',
 'V294',
 'V296',
 'V297',
 'V300',
 'V306',
 'V30

***Feature Selection***

In [16]:
useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                   'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                   'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
len(useful_features)

279

In [94]:
continuous_variables = identity_dataset_features[:11]

In [95]:
identity_dataset_continuous_variable_distribution = df_continuous_variable_summary(train_identity_df, continuous_variable)
identity_dataset_continuous_variable_distribution.to_csv('identity_dataset_continuous_variable_distribution.csv', sep='\t', index=False)

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(train)
imputed_train_df = imp_mean.transform(train)